In [ ]:
import logging

import plotly.express as px
import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import numpy as np

from core.plt_utils import plt_3d_df
from watea.watea_constants import *
from watea.energy_distribution import *
from watea.raw_watea_ts import raw_ts_it

logging.basicConfig(level=logging.INFO)


## Setup

In [ ]:
charging_points = get_preprocessed_charging_points(
    # force_update=True, 
    # force_update_extraction=True
)
cluster:DF = (
    charging_points
    .query(f"cluster_idx == {MAIN_CHARGING_REGIME_CLUSTER_IDX}")
    .query("energy_added > 320 & energy_added < 490")
    .query("current < 27.5 & current > 5.8")
    .pipe(estimate_soh)
    .query("soh >= 93 & soh <= 105")
    .eval("range_by_soc = estimated_range / soc")
)

In [ ]:
px.scatter(cluster, x="soc", y="estimated_range")

In [ ]:
plt_3d_df(cluster, "range_by_soc", "soh", "temperature", color="estimated_range", colorscale="Rainbow")

In [ ]:
cluster_charges = cluster.groupby(["id", "charge_id"]).agg({
    "odometer":"median",
    "energy_added":"median",
    "voltage":"median",
    "current":"median",
    "temperature":"median",
    "sec_duration":"median",
    "date":"median",
    "soc":"median",
    "soh":"median",
    "estimated_range": "mean",
    "estimated_range_diff": "mean",
    "range_by_soc": "median",
    #Debugging
    "id":pd.Series.mode,
    "charge_idx":pd.Series.mode,
    "charge_id":pd.Series.mode,
})

In [ ]:
plt_3d_df(cluster_charges, "soh", "range_by_soc", "temperature", color="estimated_range", colorscale="Rainbow", opacity=0.9)
